In [37]:
# Reference webiste for the model: https://towardsdatascience.com/time-series-forecasting-with-recurrent-neural-networks-74674e289816

#Imports
import pandas as pd
import numpy as np
import keras
import tensorflow as tf
from keras.preprocessing.sequence import TimeseriesGenerator
import numpy as np
import pandas as pd
from numpy.random import seed
seed(1)
from tensorflow import random
random.set_seed(2)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from pathlib import Path
%matplotlib inline
import plotly.graph_objs as go

In [64]:
#Importing CSV
csvpath = Path("Resources/USO.csv")
ETF = pd.read_csv(csvpath)
ETF.head()


,Date,open,high,low,close,volume
0,2011-05-02 00:00:00-04:00,44.99,45.6000,44.9200,44.9395,10740748
1,2011-05-03 00:00:00-04:00,44.76,44.8300,43.8000,44.1200,14207198
2,2011-05-04 00:00:00-04:00,43.92,44.0100,43.0833,43.2600,13263494
3,2011-05-05 00:00:00-04:00,41.99,42.0500,39.0200,39.3280,58446897
4,2011-05-06 00:00:00-04:00,39.24,40.6501,38.5000,38.8700,38637577


In [65]:
# Removing all columns but close_price
ETF.drop(columns=['open', 'high', 'low', 'volume'], inplace=True)
ETF.head()

,Date,close
0,2011-05-02 00:00:00-04:00,44.9395
1,2011-05-03 00:00:00-04:00,44.1200
2,2011-05-04 00:00:00-04:00,43.2600
3,2011-05-05 00:00:00-04:00,39.3280
4,2011-05-06 00:00:00-04:00,38.8700


In [66]:
#Checking For Nulls
ETF.isnull().sum()

Date     0
close    0
dtype: int64

In [67]:
#Establishing test and train data for Closing price as well as the Date
close_data = ETF['close'].values
close_data = close_data.reshape((-1,1))

In [68]:
split_percent = 0.7
split = int(split_percent*len(close_data))

In [69]:
close_train = close_data[:split]
close_test = close_data[split:]
close_train

array([[44.9395],
       [44.12  ],
       [43.26  ],
       ...,
       [11.88  ],
       [11.87  ],
       [12.17  ]])

In [70]:
date_data = ETF['Date']
date_train = ETF['Date'][:split]
date_test = ETF['Date'][split:]
print(len(close_train))
print(len(close_test))

1710
733


In [71]:
# setting the lookback, shape and 
look_back = 45

shape = (look_back,1)
# Reformat the data set to dataset of windows over a period of time.
#TimeseriesGenerator(Data, Target, sequence_length, batch)

train_generator = TimeseriesGenerator(close_train, close_train, length=look_back, batch_size=20)     
test_generator = TimeseriesGenerator(close_test, close_test, length=look_back, batch_size=1)

In [72]:
#Build LSTM Model
model = Sequential()
model.add(
    LSTM(10,
        activation='relu',
        input_shape= shape,
))
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mse')


In [73]:
# Show the model summary
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 10)                480       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 11        
Total params: 491
Trainable params: 491
Non-trainable params: 0
_________________________________________________________________


In [74]:
# Train the model
#used fit_generator because we are looking to generate new predicted data points 
num_epochs = 200
model.fit_generator(train_generator, epochs=num_epochs, verbose=1)

C:\Users\lwbak\.conda\lib\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning:

`Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.



Epoch 1/200
84/84 [==============================] - 3s 12ms/step - loss: 709.4089
Epoch 2/200
84/84 [==============================] - 1s 11ms/step - loss: 665.9787
Epoch 3/200
84/84 [==============================] - 1s 11ms/step - loss: 615.6450
Epoch 4/200
84/84 [==============================] - 1s 12ms/step - loss: 808.9296
Epoch 5/200
84/84 [==============================] - 1s 11ms/step - loss: 695.3135
Epoch 6/200
84/84 [==============================] - 1s 12ms/step - loss: 590.4684
Epoch 7/200
84/84 [==============================] - 1s 11ms/step - loss: 758.9433
Epoch 8/200
84/84 [==============================] - 1s 11ms/step - loss: 785.3809
Epoch 9/200
84/84 [==============================] - 1s 12ms/step - loss: 642.9701: 0s - loss: 6
Epoch 10/200
84/84 [==============================] - 1s 14ms/step - loss: 730.3173
Epoch 11/200
84/84 [==============================] - 1s 16ms/step - loss: 752.7487
Epoch 12/200
84/84 [==============================] - 1s 13ms/step - lo

84/84 [==============================] - 1s 13ms/step - loss: 0.2284
Epoch 100/200
84/84 [==============================] - 1s 13ms/step - loss: 0.2279
Epoch 101/200
84/84 [==============================] - 1s 12ms/step - loss: 0.2216
Epoch 102/200
84/84 [==============================] - 1s 13ms/step - loss: 0.2133
Epoch 103/200
84/84 [==============================] - 1s 15ms/step - loss: 0.1978
Epoch 104/200
84/84 [==============================] - 1s 15ms/step - loss: 0.2022
Epoch 105/200
84/84 [==============================] - 1s 15ms/step - loss: 0.2222
Epoch 106/200
84/84 [==============================] - 2s 18ms/step - loss: 0.1916
Epoch 107/200
84/84 [==============================] - 1s 16ms/step - loss: 0.2239
Epoch 108/200
84/84 [==============================] - 1s 13ms/step - loss: 0.2318
Epoch 109/200
84/84 [==============================] - 1s 12ms/step - loss: 0.2965
Epoch 110/200
84/84 [==============================] - 1s 13ms/step - loss: 0.3038
Epoch 111/200
84/8

84/84 [==============================] - 1s 15ms/step - loss: 0.2196
Epoch 197/200
84/84 [==============================] - 1s 17ms/step - loss: 0.2209
Epoch 198/200
84/84 [==============================] - 2s 19ms/step - loss: 0.2008: 0s - loss:
Epoch 199/200
84/84 [==============================] - 2s 18ms/step - loss: 0.2110
Epoch 200/200
84/84 [==============================] - 1s 16ms/step - loss: 0.2281


In [75]:
#Make predictions for futre prices
prediction = model.predict_generator(test_generator)

#Test model predictions for accuracy through plottting
close_train = close_train.reshape((-1))
close_test = close_test.reshape((-1))
prediction = prediction.reshape((-1))

#PLot for training prices
trace1 = go.Scatter(
    x = date_train,
    y = close_train,
    mode = 'lines',
    name = 'Data'
)
# PLotting our predicted prices
trace2 = go.Scatter(
    x = date_test,
    y = prediction,
    mode = 'lines',
    name = 'Prediction'
)
#plotting the actual prices for the period of time that was predicted
trace3 = go.Scatter(
    x = date_test,
    y = close_test,
    mode='lines',
    name = 'Ground Truth'
)
#Plot features and labels
layout = go.Layout(
    title = "USO",
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Close"}
)
#Show plot
fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)
fig.show()

C:\Users\lwbak\.conda\lib\site-packages\tensorflow\python\keras\engine\training.py:1905: UserWarning:

`Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.



In [76]:
#Predicting future prices

close_data = close_data.reshape((-1))

def predict(num_prediction, model):
    prediction_list = close_data[-look_back:]
    
    for _ in range(num_prediction):
        x = prediction_list[-look_back:]
        x = x.reshape((1, look_back, 1))
        out = model.predict(x)[0][0]
        prediction_list = np.append(prediction_list, out)
    prediction_list = prediction_list[look_back-1:]
        
    return prediction_list
    
def predict_dates(num_prediction):
    last_date = ETF['Date'].values[-1]
    prediction_dates = pd.date_range(last_date, periods=num_prediction+1).tolist()
    return prediction_dates

num_prediction = 90
forecast = predict(num_prediction, model)
forecast_dates = predict_dates(num_prediction)

In [77]:
forecast

array([35.76      , 35.51115036, 35.35684586, 35.13156891, 34.90761566,
       34.66237259, 34.401474  , 34.12322998, 33.82773209, 33.51510239,
       33.20019531, 32.89148712, 32.59230423, 32.30125809, 32.01638412,
       31.73749542, 31.46128082, 31.18984985, 30.91948318, 30.65147591,
       30.3825798 , 30.11452866, 29.84602737, 29.57647705, 29.30516434,
       29.03271866, 28.75809097, 28.48185158, 28.2038784 , 27.92434883,
       27.64336395, 27.36097145, 27.0772171 , 26.79211426, 26.50568199,
       26.2179184 , 25.92883682, 25.63841629, 25.34665298, 25.05354309,
       24.75900459, 24.46306801, 24.16570282, 23.86696434, 23.56684875,
       23.26546669, 22.9628582 , 22.65915871, 22.35446167, 22.04890823,
       21.7426796 , 21.43593025, 21.12884521, 20.82157707, 20.51432991,
       20.20728874, 19.90062714, 19.5944767 , 19.28897858, 18.98423576,
       18.68032837, 18.37730408, 18.07517052, 17.77388382, 17.47338104,
       17.17743301, 16.89051437, 16.61101341, 16.34051132, 16.07

In [78]:
#making a dataframe
predicted_prices = pd.DataFrame({'Date': forecast_dates, 'USO_close_price': list(forecast)}, columns=['Date', 'USO_close_price'])
predicted_prices.head()

,Date,USO_close_price
0,2021-01-13 00:00:00-05:00,35.760000
1,2021-01-14 00:00:00-05:00,35.511150
2,2021-01-15 00:00:00-05:00,35.356846
3,2021-01-16 00:00:00-05:00,35.131569
4,2021-01-17 00:00:00-05:00,34.907616


In [79]:
#export to excel
predicted_prices.to_csv(r'Resources/USO_forecast.csv')

In [61]:
# Combine all forecasted prices into a single DF
LQDpath = Path("Resources/LQD_forecast.csv")
LQD = pd.read_csv(LQDpath)
#LQD.set_index(LQD['Date'], inplace=True)

PZApath = Path("Resources/PZA_forecast.csv")
PZA = pd.read_csv(PZApath)
PZA = PZA['PZA_close_price']
#PZA.set_index(PZA['Date'], inplace=True)

USOpath = Path("Resources/USO_forecast.csv")
USO = pd.read_csv(USOpath)
USO = USO['USO_close_price']
#USO.set_index(USO['Date'], inplace=True)

TLTpath = Path("Resources/TLT_forecast.csv")
TLT = pd.read_csv(TLTpath)
TLT = TLT['TLT_close_price']
#TLT.set_index(TLT['Date'], inplace=True)

QQQpath = Path("Resources/QQQ_forecast.csv")
QQQ = pd.read_csv(QQQpath)
QQQ = QQQ['QQQ_close_price']
#QQQ.set_index(QQQ['Date'], inplace=True)

VMBSpath = Path("Resources/VMBS_forecast.csv")
VMBS = pd.read_csv(VMBSpath)
VMBS = VMBS['VMBS_close_price']
#VMBS.set_index(VMBS['Date'], inplace=True)

VNQpath = Path("Resources/VNQ_forecast.csv")
VNQ = pd.read_csv(VNQpath)
VNQ = VNQ['VNQ_close_price']

USO.head()

0    130.800000
1    126.370728
2    131.667343
3    131.501678
4    135.907135
Name: QQQ_close_price, dtype: float64

In [62]:
#Concatenate and clean the data
forecast_prices = pd.concat([LQD, PZA, USO, TLT, QQQ, VMBS, VNQ], axis='columns', join='inner')
forecast_prices = forecast_prices.drop(columns=["Unnamed: 0"])
forecast_prices.head()

,Date,LQD_close_price,PZA_close_price,USO_close_price,TLT_close_price,QQQ_close_price,VMBS_close_price,VNQ_close_price
0,2021-01-13 00:00:00-05:00,136.520000,27.090000,35.760000,152.360000,130.800000,54.010000,84.310000
1,2021-01-14 00:00:00-05:00,137.277298,27.168526,35.348991,154.565033,126.370728,54.105686,81.114426
2,2021-01-15 00:00:00-05:00,138.050400,27.249393,35.693897,155.340942,131.667343,54.160480,80.127251
3,2021-01-16 00:00:00-05:00,138.771729,27.327137,35.377987,157.228912,131.501678,54.233402,80.206505
4,2021-01-17 00:00:00-05:00,139.553513,27.401978,35.770760,157.639771,135.907135,54.292553,80.162140


In [63]:
#EXPORT EXCEL
forecast_prices.to_csv(r'Resources/forecast_prices.csv')